<a href="https://colab.research.google.com/github/Arpita2512/DPhi_Data_Sprint_50/blob/main/Credit_Risk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/My Drive/dphi credit risk')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Basic EDA

In [ ]:
# explore dataset

import numpy as np
import pandas as pd

train = pd.read_csv('train_dataset.csv', index_col=0)
train.head()

,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,installment_commitment,personal_status,other_parties,residence_since,property_magnitude,age,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker,class
0,'no checking',18,'critical/other existing credit',radio/tv,1800,'<100','1<=X<4',4,'male single',none,2,car,24,none,own,2,skilled,1,none,yes,1
1,'<0',24,'existing paid',radio/tv,2439,'<100','<1',4,'female div/dep/mar',none,4,'real estate',35,none,own,1,skilled,1,yes,yes,0
2,'no checking',36,'no credits/all paid',repairs,2613,'<100','1<=X<4',4,'male single',none,2,car,27,none,own,2,skilled,1,none,yes,1
3,'no checking',15,'existing paid',education,4623,'100<=X<500','1<=X<4',3,'male single',none,2,'life insurance',40,none,own,1,'high qualif/self emp/mgmt',1,yes,yes,0
4,'<0',21,'existing paid',furniture/equipment,3599,'<100','4<=X<7',1,'female div/dep/mar',none,4,car,26,none,rent,1,'unskilled resident',1,none,yes,1


In [ ]:
train.isnull().sum()

checking_status           0
duration                  0
credit_history            0
purpose                   0
credit_amount             0
savings_status            0
employment                0
installment_commitment    0
personal_status           0
other_parties             0
residence_since           0
property_magnitude        0
age                       0
other_payment_plans       0
housing                   0
existing_credits          0
job                       0
num_dependents            0
own_telephone             0
foreign_worker            0
class                     0
dtype: int64

In [ ]:
train.dtypes

checking_status           object
duration                   int64
credit_history            object
purpose                   object
credit_amount              int64
savings_status            object
employment                object
installment_commitment     int64
personal_status           object
other_parties             object
residence_since            int64
property_magnitude        object
age                        int64
other_payment_plans       object
housing                   object
existing_credits           int64
job                       object
num_dependents             int64
own_telephone             object
foreign_worker            object
class                      int64
dtype: object

In [ ]:
# explore distinct values for each column

for col in train:
  print(col, train[col].unique(), '\n')

checking_status ["'no checking'" "'<0'" "'0<=X<200'" "'>=200'"] 

duration [18 24 36 15 21 12 48  9  4 45 30  7  6 42  8 10 39 20  5 27 11 60 33 14
 28 13 40 47 22 16 54 26] 

credit_history ["'critical/other existing credit'" "'existing paid'"
 "'no credits/all paid'" "'all paid'" "'delayed previously'"] 

purpose ['radio/tv' 'repairs' 'education' 'furniture/equipment' 'business'
 "'used car'" "'new car'" 'retraining' 'other' "'domestic appliance'"] 

credit_amount [ 1800  2439  2613  4623  3599  5801  2659  4844  7814  2462  3617  6758
  3621   458  2923  3590  1131  1544  2825  1845  6948  1240  1037  9960
  1474  1295  1313  1842  2910  2022  1239  4811  4280   339  3181   730
  4308  1740 12612  2375   428  2991   959  6568  1919  1200  2600  1409
  4351  1498  1449  1282  4675  4370  5096  1526  4473   590  1414  3835
  1533   929   683  3914  1249  2647  6615  3398  8072  1330  1264  7166
  1103  5771  2503   571  6199  1352  1403  3399  3578   781  2141  8588
  1908  1164  1928

# Preprocessing

In [ ]:
# extract relevant features

# begin with all features

features = train[
             list(train.columns)     
].copy()

In [ ]:
# columns removed iteratively using permutation importance (see below)

numerical_cols = [
                  'duration', 'credit_amount', 'installment_commitment',
                  'residence_since', 'age', 'existing_credits'#, 'num_dependents'
]

In [ ]:
# columns removed iteratively using permutation importance (see below)

categorical_cols = [
                'checking_status', 'credit_history', 'purpose', 'savings_status',
                'employment', 'personal_status', 'other_parties', 'property_magnitude',
                'housing' 
]
#'other_payment_plans', 'own_telephone', 'job', 'foreign_worker'

In [ ]:
# no null values so no imputation required

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler

numerical_transformer = Pipeline(steps=[
    ('scaler', MinMaxScaler())
])

categorical_transformer = Pipeline(steps=[
    ('encode', OneHotEncoder())
])

# define preprocessing steps

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

In [ ]:
# split data into train and test sets
all_features = numerical_cols + categorical_cols
from sklearn.model_selection import train_test_split
x = features[all_features]
y =  features['class']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=0)
x_train.shape, x_test.shape

((720, 15), (80, 15))

# Build Model

In [ ]:
!pip install catboost

In [ ]:
from sklearn.metrics import f1_score
from catboost import CatBoostClassifier

# Build pipeline and fit data

my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', CatBoostClassifier(random_state=0))
                             ])

 
my_pipeline.fit(x_train, y_train)

Learning rate set to 0.008954
0:	learn: 0.6897328	total: 1.39ms	remaining: 1.39s
1:	learn: 0.6870107	total: 2.61ms	remaining: 1.3s
2:	learn: 0.6833029	total: 3.74ms	remaining: 1.24s
3:	learn: 0.6798302	total: 4.89ms	remaining: 1.22s
4:	learn: 0.6762211	total: 6.12ms	remaining: 1.22s
5:	learn: 0.6732876	total: 8.12ms	remaining: 1.34s
6:	learn: 0.6701127	total: 10.3ms	remaining: 1.46s
7:	learn: 0.6675607	total: 12.4ms	remaining: 1.54s
8:	learn: 0.6644612	total: 14.6ms	remaining: 1.61s
9:	learn: 0.6615764	total: 17ms	remaining: 1.68s
10:	learn: 0.6585214	total: 19.2ms	remaining: 1.72s
11:	learn: 0.6554156	total: 21.3ms	remaining: 1.75s
12:	learn: 0.6523562	total: 23.5ms	remaining: 1.78s
13:	learn: 0.6498048	total: 25.7ms	remaining: 1.81s
14:	learn: 0.6472873	total: 27.8ms	remaining: 1.83s
15:	learn: 0.6450054	total: 29.9ms	remaining: 1.84s
16:	learn: 0.6421454	total: 32ms	remaining: 1.85s
17:	learn: 0.6394001	total: 34.1ms	remaining: 1.86s
18:	learn: 0.6369609	total: 36.1ms	remaining: 1.8

Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('num',
                                                  Pipeline(memory=None,
                                                           steps=[('scaler',
                                                                   MinMaxScaler(copy=True,
                                                                                feature_range=(0,
                                                                                               1)))],
                                                           verbose=False),
                                                  ['duration', 'credit_amount',
                                                   'installment_commitment',
                                

In [ ]:
# get accuracy scores

my_pipeline.score(x_train, y_train), my_pipeline.score(x_test, y_test)

(0.9722222222222222, 0.825)

In [ ]:
# get F1 Score (contest metric)

preds = my_pipeline.predict(x_test)

score = f1_score(y_test, preds)
print(score)

0.8833333333333333


# Permuation Importance

In [ ]:
# run iteratively to remove feature having no/negative impact on model performance

from sklearn.inspection import permutation_importance

result = permutation_importance(my_pipeline, x_test, y_test,
                                random_state=0)

In [ ]:
# PI scores corresponding to each column

result['importances_mean']

array([0.085 , 0.0575, 0.0575, 0.015 , 0.065 , 0.01  , 0.0975, 0.055 ,
       0.03  , 0.03  , 0.0425, 0.0625, 0.005 , 0.055 , 0.01  ])

In [ ]:
x_test.columns

Index(['duration', 'credit_amount', 'installment_commitment',
       'residence_since', 'age', 'existing_credits', 'checking_status',
       'credit_history', 'purpose', 'savings_status', 'employment',
       'personal_status', 'other_parties', 'property_magnitude', 'housing'],
      dtype='object')

# Predictions

In [ ]:
# repeat all train set steps

test = pd.read_csv('test_data.csv')

In [ ]:
features = test[list(test.columns)].copy()

In [ ]:
# store predictions as dataframe

submission = pd.DataFrame(my_pipeline.predict(features[all_features]))
submission.index = test.index
submission.columns = ['prediction']
submission.head()

,prediction
0,1
1,1
2,1
3,1
4,0


In [ ]:
# save as csv

submission.to_csv('submission.csv', index=False)